In [ ]:
import numpy as np
import sep

In [ ]:
import astropy
import matplotlib.pyplot as plt
from matplotlib import rcParams

%matplotlib inline

rcParams['figure.figsize'] = [10., 8.]

In [ ]:
# read image into standard 2-d numpy array
data = astropy.io.fits.get_data("hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits")

In [ ]:
# show the image
m, s = np.mean(data), np.std(data)
plt.imshow(data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar();
plt.savefig("MyFigure.png", format='png')

**Background subtraction**

In [ ]:
# measure a spatially varying background on the image
bkg = sep.Background(data)

In [ ]:
bkg = sep.Background(data, mask=mask, bw=64, bh=64, fw=3, fh=3)

In [ ]:
# get a "global" mean and noise of the image background:
print(bkg.globalback)
print(bkg.globalrms)

In [ ]:
# evaluate background as 2-d array, same size as original image
bkg_image = bkg.back()
# bkg_image = np.array(bkg) # equivalent to above

In [ ]:
# show the background
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.savefig("MyFigure.png", format='png')

In [ ]:
# evaluate the background noise as 2-d array, same size as original image
bkg_rms = bkg.rms()

In [ ]:
# show the background noise
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.savefig("MyFigure.png", format='png')

In [ ]:
# subtract the background
data_sub = data - bkg

**Object detection**

In [ ]:
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)

In [ ]:
# how many objects were detected
len(objects)

In [ ]:
from matplotlib.patches import Ellipse

# plot background-subtracted image
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray',
               vmin=m-s, vmax=m+s, origin='lower')

# plot an ellipse for each object
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
                width=6*objects['a'][i],
                height=6*objects['b'][i],
                angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)
plt.savefig("MyFigure.png", format='png')

In [ ]:
# available fields
objects.dtype.names

In [ ]:
len(object) 
print(len(object))


**Aperture photometry**

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'],
                                     3.0, err=bkg.globalrms, gain=1.0)

In [ ]:
matplotlib.pyplot.hist(flux)
print(matplotlib.pyplot.hist(flux))

mean=numpy.mean(flux)
print(numpy.mean(flux))

median=numpy.median(flux)
print(numpy.median(flux))

std=numpy.std(flux)
print(numpy.std(flux)

In [ ]:
thedifference = flux - numpy.mean(flux)

outlier_idx = numpy.argmax(thedifference)



In [ ]:
from matplotlib.patches import Ellipse

# plot background-subtracted image
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray',
               vmin=m-s, vmax=m+s, origin='lower')

# plot an ellipse for each object

e = Ellipse(xy=(objects['x'][outlier_idx], objects['y'][outlier_idx]),width=6*objects['a'][outlier_idx],height=6*objects['b'][outlier_idx],angle=objects['theta'][outlier_idx] * 180. / np.pi)
e.set_facecolor('none')
e.set_edgecolor('red')
ax.add_artist(e)
plt.savefig("MyFigure.png", format='png')

In [ ]:
# show the first 10 objects results:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))

**Point 8 3-color face image**

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
f105w = fits.getdata('f105w.fits')
f125w = fits.getdata('f125w.fits')
f160w = fits.getdata('f160w.fits')

In [ ]:
median_f105w = np.nanmedian(f105w[f105w<1])
median_f125w = np.nanmedian(f125w[f125w<1])
median_f160w = np.nanmedian(f160w[f160w<1])
print(f'Median of F105W sky {median_f105w}')
print(f'Median of F125W sky {median_f125w}')
print(f'Median of F160W sky {median_f160w}')

In [ ]:
f105w = f105w.byteswap().newbyteorder()
f125w = f125w.byteswap().newbyteorder()
f160w = f160w.byteswap().newbyteorder()

bkg_f105w = sep.Background(f105w)
bkg_f125w = sep.Background(f125w)
bkg_f160w = sep.Background(f160w)

f105w -= bkg_f105w
f125w -= bkg_f125w
f160w -= bkg_f160w

In [ ]:
median_f105w = np.nanmedian(f105w[f105w<1])
median_f125w = np.nanmedian(f125w[f125w<1])
median_f160w = np.nanmedian(f160w[f160w<1])

print(f'Median of F105W sky {median_f105w}')
print(f'Median of F125W sky {median_f125w}')
print(f'Median of F160W sky {median_f160w}')

In [ ]:
def rescale_data(data,dmin=0.01,dmax=10.0):

    datac = np.clip(data,dmin,dmax)
    return (np.log10(dtac)-np.log10(dmin)/(np.log10(dmax)-np.log10(dmin)))

In [ ]:
p_f105w = rescale_data(f105w)
p_f125w = rescale_data(f125w)
p_f160w = rescale_data(f160w)

In [ ]:
rgb = np.stack([p_f160w,0.7*p_f125w,p_105w],axis=-1)
print(rgb.shape)

In [ ]:
xsize = rgb.shape[1]
ysize = rgb.shape[0]

f,ax = plt.subplots(1,1,figsize=(xsize/1000.,ysize/1000.0))
ax.axis('off')
ax.imshow(rgb,orgin='lower')

In [ ]:
plt.imsave('smacs-0723.png',rgb,origin='lower')

In [ ]:
objects = sep.extract(f160w, 5, err=bkgf160w.globalrms)